In [ ]:
#Importing required libraries
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random

Don't forget to add instructions for replication

In [1]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Force CPU for maximum consistency if preferred
device = torch.device("cpu") 


NameError: name 'np' is not defined

Do not import tensorflow and pytorch in the same notebook as it crashes the notebook

# Dataset 1: Bike Sharing

### Characteristics

In [ ]:
# fetch dataset 
bike_sharing = fetch_ucirepo(id=275) 
  
# data (as pandas dataframes) 
X_bike = bike_sharing.data.features 
y_bike = bike_sharing.data.targets 

bike_df = pd.concat([X_bike, y_bike], axis=1)


Characteristic 1 & 2.

In [ ]:
num_variables = bike_df.shape[1]
num_records = bike_df.shape[0]
print(f"Number of variables (columns): {num_variables}")
print(f"Number of records (rows): {num_records}\n")

Number of variables (columns): 14
Number of records (rows): 17379



Characteristic 3.

In [ ]:
bike_df.head(10)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,1
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,1
6,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2
7,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,3
8,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,8
9,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,14


In [ ]:
bike_df.tail(10)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
17369,2012-12-31,1,1,12,14,0,1,1,2,0.28,0.2727,0.45,0.2239,247
17370,2012-12-31,1,1,12,15,0,1,1,2,0.28,0.2879,0.45,0.1343,315
17371,2012-12-31,1,1,12,16,0,1,1,2,0.26,0.2576,0.48,0.1940,214
17372,2012-12-31,1,1,12,17,0,1,1,2,0.26,0.2879,0.48,0.0896,164
17373,2012-12-31,1,1,12,18,0,1,1,2,0.26,0.2727,0.48,0.1343,122
17374,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,119
17375,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,89
17376,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,90
17377,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,61
17378,2012-12-31,1,1,12,23,0,1,1,1,0.26,0.2727,0.65,0.1343,49


In [ ]:
print("Data types of each column:")
print(bike_df.dtypes)
print("\n")

Data types of each column:
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
cnt             int64
dtype: object




| Column      | Type         | Subtype            | Explanation |
|-------------|--------------|--------------------|-------------|
| index       | Categorical  | Nominal            | Row identifier with no numeric meaning |
| dteday      | Categorical  | Nominal (date)     | Treated as a label unless decomposed |
| season      | Categorical  | Ordinal            | Encoded seasons with natural order (1–4) |
| yr          | Categorical  | Ordinal            | 0 = 2011, 1 = 2012 (progression in time) |
| mnth        | Categorical  | Ordinal            | Months have natural order (1–12) |
| hr          | Categorical  | Ordinal            | Hours of day (0–23) with natural order |
| holiday     | Categorical  | Binary Nominal     | Holiday flag (0/1) |
| weekday     | Categorical  | Nominal            | Days of week (0–6), no natural order |
| workingday  | Categorical  | Binary Nominal     | Working day flag (0/1) |
| weathersit  | Categorical  | Ordinal            | Weather categories ranked by severity (1–4) |
| temp        | Numerical    | Continuous         | Normalized temperature |
| atemp       | Numerical    | Continuous         | Normalized “feels‑like” temperature |
| hum         | Numerical    | Continuous         | Normalized humidity |
| windspeed   | Numerical    | Continuous         | Normalized windspeed |
| cnt         | Numerical    | Discrete           | Count of total bike rentals |


Characteristic 4.

In [ ]:

# Generate descriptive (summary) statistics for bike_df.
print("Descriptive (summary) statistics:")
print(bike_df.describe())
print("\n")

Descriptive (summary) statistics:
             season            yr          mnth            hr       holiday  \
count  17379.000000  17379.000000  17379.000000  17379.000000  17379.000000   
mean       2.501640      0.502561      6.537775     11.546752      0.028770   
std        1.106918      0.500008      3.438776      6.914405      0.167165   
min        1.000000      0.000000      1.000000      0.000000      0.000000   
25%        2.000000      0.000000      4.000000      6.000000      0.000000   
50%        3.000000      1.000000      7.000000     12.000000      0.000000   
75%        3.000000      1.000000     10.000000     18.000000      0.000000   
max        4.000000      1.000000     12.000000     23.000000      1.000000   

            weekday    workingday    weathersit          temp         atemp  \
count  17379.000000  17379.000000  17379.000000  17379.000000  17379.000000   
mean       3.003683      0.682721      1.425283      0.496987      0.475775   
std        2.0057

Characteristics 5

In [ ]:
# Check for and print the total count of missing values for each column in bike_df.
missing_values = bike_df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("No missing values found.\n")
else:
    print("\n")

# Check for and print the total count of duplicate rows in bike_df.
duplicate_rows = bike_df.duplicated().sum()
print(f"Total count of duplicate rows: {duplicate_rows}\n")

# Print the column names of bike_df to identify potentially irrelevant columns.
print("Column names in bike_df:")
print(bike_df.columns.tolist())
print("\n")


Missing values per column:
Series([], dtype: int64)
No missing values found.

Total count of duplicate rows: 0

Column names in bike_df:
['dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'cnt']




Characteristics 6.

Balancing assessment: The target variable 'cnt' is a continuous numerical variable representing bike rental counts. This indicates a regression problem, not a classification problem. Therefore, dataset balancing (e.g., handling class imbalance) is not relevant for this task.

In [ ]:
# this columns where identified thanks to the summary statistics

numerical_features = ['temp', 'atemp', 'hum', 'windspeed']
ordinal_features = ['hr', 'mnth', 'season', 'weathersit']
binary_features = ['holiday', 'workingday']  # do NOT scale
target = ['cnt']  # regression target

# Combine features to scale
features_to_scale = numerical_features + ordinal_features

scaler = MinMaxScaler()

bike_scaled_df = bike_df.copy()
bike_scaled_df[features_to_scale] = scaler.fit_transform(bike_df[features_to_scale])

bike_scaled_df['cnt'] = bike_df['cnt']

bike_scaled_df.head()


,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01,0.0,0,0.0,0.000000,0,6,0,0.0,0.224490,0.2879,0.81,0.0,16
1,2011-01-01,0.0,0,0.0,0.043478,0,6,0,0.0,0.204082,0.2727,0.80,0.0,40
2,2011-01-01,0.0,0,0.0,0.086957,0,6,0,0.0,0.204082,0.2727,0.80,0.0,32
3,2011-01-01,0.0,0,0.0,0.130435,0,6,0,0.0,0.224490,0.2879,0.75,0.0,13
4,2011-01-01,0.0,0,0.0,0.173913,0,6,0,0.0,0.224490,0.2879,0.75,0.0,1


Characteristic 7.

In [ ]:
train_size = int(0.70 * num_records)
test_size = num_records - train_size
print(f"If a 70/30 split were performed:")
print(f"  Training set records (70%): {train_size}")
print(f"  Test set records (30%): {test_size}")

If a 70/30 split were performed:
  Training set records (70%): 12165
  Test set records (30%): 5214


### Dataset preprocessing:
* Convert to supervised learning (sliding window)
* Create sequences (e.g., 24‑hour window → next hour prediction)

In [ ]:
# Convert scaled bike dataframe to numpy array
bike_scaled_df.drop(columns=['dteday'], inplace=True)
bike_data = bike_scaled_df.values

lookback_window = 24

# Create sequences using sliding window
X_bike_seq, y_bike_seq = [], []

for i in range(len(bike_data) - lookback_window):
    X_bike_seq.append(bike_data[i:i + lookback_window, :-1])
    # Target: next hour's bike count (cnt)
    y_bike_seq.append(bike_data[i + lookback_window, -1])

X_bike_seq = np.array(X_bike_seq)
y_bike_seq = np.array(y_bike_seq)

print(f"Sequence shape - X: {X_bike_seq.shape}, y: {y_bike_seq.shape}")
print(f"Number of sequences: {X_bike_seq.shape[0]}")
print(f"Time steps per sequence: {X_bike_seq.shape[1]}")
print(f"Number of features: {X_bike_seq.shape[2]}\n")


Sequence shape - X: (17355, 24, 12), y: (17355,)
Number of sequences: 17355
Time steps per sequence: 24
Number of features: 12



The Bike Sharing dataset is highly cyclical based on daily routines (commutes, school hours). A 24-hour window allows the RNN and 1D-CNN to "see" a full day's progression, capturing the relationship between the current hour and the same hour from the previous day, which is the strongest seasonal indicator in this data.

The dteday column is dropped because it contains string-formatted timestamps which are incompatible with the mathematical operations in PyTorch and TensorFlow. This does not affect the model's performance because the temporal information is already preserved through the numeric features yr, mnth, hr, holiday, weekday, and workingday. Dropping the raw string ensures the dataset is a purely numeric $float32$ matrix, which is required for tensor conversion.

In [ ]:
# Split into train and test sets (70/30 split)
train_size = int(0.70 * len(X_bike_seq))
test_size = len(X_bike_seq) - train_size

X_bike_train = X_bike_seq[:train_size]
y_bike_train = y_bike_seq[:train_size]

X_bike_test = X_bike_seq[train_size:]
y_bike_test = y_bike_seq[train_size:]

print(f"Training set:")
print(f"  X shape: {X_bike_train.shape}, y shape: {y_bike_train.shape}")
print(f"Training samples: {X_bike_train.shape[0]}")
print(f"\nTest set:")
print(f"  X shape: {X_bike_test.shape}, y shape: {y_bike_test.shape}")
print(f"Test samples: {X_bike_test.shape[0]}\n")

Training set:
  X shape: (12148, 24, 12), y shape: (12148,)
Training samples: 12148

Test set:
  X shape: (5207, 24, 12), y shape: (5207,)
Test samples: 5207



In [ ]:
# for pythorch implementations:
X_bike_train_torch = torch.FloatTensor(X_bike_train)
y_bike_train_torch = torch.FloatTensor(y_bike_train)
X_bike_test_torch = torch.FloatTensor(X_bike_test)
y_bike_test_torch = torch.FloatTensor(y_bike_test)

print("PyTorch tensors created:")
print(f"  X_bike_train_torch: {X_bike_train_torch.shape}")
print(f"  y_bike_train_torch: {y_bike_train_torch.shape}\n")

PyTorch tensors created:
  X_bike_train_torch: torch.Size([12148, 24, 12])
  y_bike_train_torch: torch.Size([12148])



## Technique 1: Vanilla RNN

In [ ]:
train_ds = TensorDataset(X_bike_train_torch, y_bike_train_torch)
test_ds = TensorDataset(X_bike_test_torch, y_bike_test_torch)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=False) # Shuffle=False for replicability
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [ ]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # out: (batch, seq_len, hidden_size)
        out, _ = self.rnn(x)
        # We only care about the last time step's prediction
        return self.fc(out[:, -1, :])

In [ ]:
rnn_model = VanillaRNN(input_size=12, hidden_size=64, output_size=1).to(device)
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(15):
    rnn_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        loss = criterion(rnn_model(batch_X).squeeze(), batch_y)
        loss.backward()
        optimizer.step()

In [ ]:
rnn_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        preds = rnn_model(batch_X)
        y_pred.extend(preds.squeeze().tolist())
        y_true.extend(batch_y.tolist())

y_true, y_pred = np.array(y_true), np.array(y_pred)

print("--- Vanilla RNN Metrics ---")
print(f"R^2 Score: {r2_score(y_true, y_pred):.4f}")
print(f"MSE:       {mean_squared_error(y_true, y_pred):.4f}")
print(f"RMSE:      {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
print(f"MAE:       {mean_absolute_error(y_true, y_pred):.4f}")

## Technique 2: RestNet

In [ ]:
train_ds = TensorDataset(X_bike_train_torch, y_bike_train_torch)
test_ds = TensorDataset(X_bike_test_torch, y_bike_test_torch)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=False) # Shuffle=False for replicability
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [ ]:
class ResBlock1D(nn.Module):
    def __init__(self, channels):
        super(ResBlock1D, self).__init__()
        self.conv = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.bn(self.conv(x)))
        out = self.bn(self.conv(out))
        return self.relu(out + residual) # The ResNet Skip Connection

class ResNet1D(nn.Module):
    def __init__(self, input_dim, seq_len):
        super(ResNet1D, self).__init__()
        # Conv1d expects (Batch, Channels, Seq), so we transpose in forward()
        self.init_conv = nn.Conv1d(input_dim, 32, kernel_size=3, padding=1)
        self.res_layers = nn.Sequential(ResBlock1D(32), ResBlock1D(32))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * seq_len, 1)

    def forward(self, x):
        x = x.transpose(1, 2) # Rearrange to (Batch, Features, Seq)
        x = torch.relu(self.init_conv(x))
        x = self.res_layers(x)
        x = self.flatten(x)
        return self.fc(x)

In [ ]:
resnet_model = ResNet1D(input_dim=12, seq_len=24).to(device)
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(15):
    resnet_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        loss = criterion(resnet_model(batch_X).squeeze(), batch_y)
        loss.backward()
        optimizer.step()

In [ ]:
resnet_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        preds = resnet_model(batch_X)
        y_pred.extend(preds.squeeze().tolist())
        y_true.extend(batch_y.tolist())

y_true, y_pred = np.array(y_true), np.array(y_pred)

print("--- 1D-CNN ResNet Metrics ---")
print(f"R^2 Score: {r2_score(y_true, y_pred):.4f}")
print(f"MSE:       {mean_squared_error(y_true, y_pred):.4f}")
print(f"RMSE:      {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
print(f"MAE:       {mean_absolute_error(y_true, y_pred):.4f}")

# Dataset 2: Individual Household Electric Power Consumption

### Characteristics

In [ ]:
# fetch dataset
individual_household_electric_power_consumption = fetch_ucirepo(id=235)

# data (as pandas dataframes)
electric_X = individual_household_electric_power_consumption.data.features
electric_y = individual_household_electric_power_consumption.data.targets
electric_df = pd.concat([electric_X, electric_y], axis=1)

/opt/miniconda3/envs/CA1NN/lib/python3.10/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


Characteristic 1 & 2.

In [ ]:
num_variables = electric_df.shape[1]
num_records = electric_df.shape[0]
print(f"Number of variables (columns): {num_variables}")
print(f"Number of records (rows): {num_records}\n")

Number of variables (columns): 9
Number of records (rows): 2075259



Characteristic 3.

In [ ]:
electric_df.head(10)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
5,16/12/2006,17:29:00,3.520,0.522,235.020,15.000,0.000,2.000,17.0
6,16/12/2006,17:30:00,3.702,0.520,235.090,15.800,0.000,1.000,17.0
7,16/12/2006,17:31:00,3.700,0.520,235.220,15.800,0.000,1.000,17.0
8,16/12/2006,17:32:00,3.668,0.510,233.990,15.800,0.000,1.000,17.0
9,16/12/2006,17:33:00,3.662,0.510,233.860,15.800,0.000,2.000,16.0


In [ ]:
print("Data types of each column:")
print(electric_df.dtypes)
print("\n")

Data types of each column:
Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object




some columns are set to objects, rather than a numerical type, due to this they have to be parse to they can be taken into account as the type of data they actually are

In [ ]:
numeric_cols = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]
electric_df[numeric_cols] = electric_df[numeric_cols].apply(
    pd.to_numeric,
    errors='coerce'  # invalid values (e.g. '?') become NaN
)

| Column                | Type        | Subtype        | Explanation |
|-----------------------|-------------|----------------|-------------|
| Date                  | Categorical | Nominal        | Calendar date label, no numeric meaning |
| Time                  | Categorical | Nominal        | Time-of-day label, no numeric meaning |
| Global_active_power   | Numerical   | Continuous     | Real-valued household active power consumption (kW) |
| Global_reactive_power | Numerical   | Continuous     | Real-valued reactive power consumption (kW) |
| Voltage               | Numerical   | Continuous     | Real-valued voltage measurement (volts) |
| Global_intensity      | Numerical   | Continuous     | Real-valued current intensity (amps) |
| Sub_metering_1        | Numerical   | Discrete       | Integer energy consumption count for kitchen appliances |
| Sub_metering_2        | Numerical   | Discrete       | Integer energy consumption count for laundry appliances |
| Sub_metering_3        | Numerical   | Discrete       | Integer energy consumption count for water heater & AC |


Characteristic 4.

In [ ]:

# Generate descriptive (summary) statistics for electric_df.
print("Descriptive (summary) statistics:")
print(electric_df.describe())
print("\n")

Descriptive (summary) statistics:
       Global_active_power  Global_reactive_power       Voltage  \
count         2.049280e+06           2.049280e+06  2.049280e+06   
mean          1.091615e+00           1.237145e-01  2.408399e+02   
std           1.057294e+00           1.127220e-01  3.239987e+00   
min           7.600000e-02           0.000000e+00  2.232000e+02   
25%           3.080000e-01           4.800000e-02  2.389900e+02   
50%           6.020000e-01           1.000000e-01  2.410100e+02   
75%           1.528000e+00           1.940000e-01  2.428900e+02   
max           1.112200e+01           1.390000e+00  2.541500e+02   

       Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
count      2.049280e+06    2.049280e+06    2.049280e+06    2.049280e+06  
mean       4.627759e+00    1.121923e+00    1.298520e+00    6.458447e+00  
std        4.444396e+00    6.153031e+00    5.822026e+00    8.437154e+00  
min        2.000000e-01    0.000000e+00    0.000000e+00    0.00000

Characteristics 5

In [ ]:
# Check for and print the total count of missing values for each column in electric_df.
missing_values = electric_df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("No missing values found.\n")
else:
    print("\n")

# Check for and print the total count of duplicate rows in electric_df.
duplicate_rows = electric_df.duplicated().sum()
print(f"Total count of duplicate rows: {duplicate_rows}\n")

# Print the column names of electric_df to identify potentially irrelevant columns.
print("Column names in electric_df:")
print(electric_df.columns.tolist())
print("\n")


Missing values per column:
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Total count of duplicate rows: 0

Column names in electric_df:
['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']




Characteristics 6.

Balancing assessment: The target variable 'Global_active_power' is a continuous numerical variable representing household power consumption in kilowatts. This indicates a regression problem, not a classification problem. Therefore, dataset balancing (e.g., handling class imbalance) is not relevant for this task.

In [ ]:
# this columns where identified thanks to the summary statistics

numeric_features = [ 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3' ]


scaler = MinMaxScaler()

electric_scaled_df = electric_df.copy()
electric_scaled_df[numeric_features] = scaler.fit_transform(electric_df[numeric_features])

electric_scaled_df['Global_active_power'] = electric_df['Global_active_power']

electric_scaled_df.head()


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.300719,0.376090,0.377593,0.0,0.0125,0.548387
1,16/12/2006,17:25:00,5.360,0.313669,0.336995,0.473029,0.0,0.0125,0.516129
2,16/12/2006,17:26:00,5.374,0.358273,0.326010,0.473029,0.0,0.0250,0.548387
3,16/12/2006,17:27:00,5.388,0.361151,0.340549,0.473029,0.0,0.0125,0.548387
4,16/12/2006,17:28:00,3.666,0.379856,0.403231,0.323651,0.0,0.0125,0.548387


Characteristic 7.

In [ ]:
train_size = int(0.70 * num_records)
test_size = num_records - train_size
print(f"If a 70/30 split were performed:")
print(f"  Training set records (70%): {train_size}")
print(f"  Test set records (30%): {test_size}")

If a 70/30 split were performed:
  Training set records (70%): 1452681
  Test set records (30%): 622578


### Dataset preprocessing:
* Resample to hourly
* Create sequences

note: I will be not as verbose as in the previous dataset as since most check and justification still hold on this one

In [ ]:
datetime_str = electric_scaled_df['Date'] + ' ' + electric_scaled_df['Time']
electric_scaled_df['DateTime'] = pd.to_datetime(datetime_str, dayfirst=True)

electric_scaled_df.set_index('DateTime', inplace=True)
df_numeric = electric_scaled_df.select_dtypes(include=[np.number])

# Using mean() to aggregate the minutes into an hour
df_hourly = df_numeric.resample('H').mean()

# Handle Potential Missing Values (Common after resampling)
df_hourly = df_hourly.ffill() 

# Feature Engineering: Replicate the 'Time Context' 
df_hourly['hour'] = df_hourly.index.hour
df_hourly['day_of_week'] = df_hourly.index.dayofweek
df_hourly['month'] = df_hourly.index.month

# Statement: We drop the DateTime index here as yr/mnth/hr features now represent time.
power_data = df_hourly.values.astype(np.float32)

# Create Sequences (Sliding Window)
lookback_window = 24 # 24 hours
X_list, y_list = [], []

for i in range(len(power_data) - lookback_window):
    X_list.append(power_data[i : i + lookback_window, 1:]) 
    y_list.append(power_data[i + lookback_window, 0])

X_power_seq = np.array(X_list)
y_power_seq = np.array(y_list)

/var/folders/s7/06176ck52670b55rv_lly5980000gn/T/ipykernel_56542/2462929361.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df_numeric.resample('H').mean()


In [ ]:
# Split and Convert
train_size = int(0.70 * len(X_power_seq))

print(f"Resampled Shape: {df_hourly.shape}")
print(f"Sequence Shape: {X_power_seq.shape}")

Resampled Shape: (34589, 10)
Sequence Shape: (34565, 24, 9)


In [ ]:
# PyTorch
X_train_torch = torch.from_numpy(X_power_seq[:train_size])
y_train_torch = torch.from_numpy(y_power_seq[:train_size])


## Technique 1: Vanilla RNN

In [ ]:
X_test_torch = torch.from_numpy(X_power_seq[train_size:])
y_test_torch = torch.from_numpy(y_power_seq[train_size:])

train_loader = DataLoader(TensorDataset(X_train_torch, y_train_torch), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_torch, y_test_torch), batch_size=64, shuffle=False)

In [ ]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])

In [ ]:
rnn_model = VanillaRNN(input_size=9, hidden_size=64, output_size=1).to(device)
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("Training Vanilla RNN on Power Dataset...")
for epoch in range(15):
    rnn_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        loss = criterion(rnn_model(batch_X).squeeze(), batch_y)
        loss.backward()
        optimizer.step()

In [ ]:
rnn_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        preds = rnn_model(batch_X)
        y_pred.extend(preds.squeeze().tolist())
        y_true.extend(batch_y.tolist())

y_true, y_pred = np.array(y_true), np.array(y_pred)

print("\n--- Vanilla RNN Metrics (Electric Power) ---")
print(f"R^2 Score: {r2_score(y_true, y_pred):.4f}")
print(f"MSE:       {mean_squared_error(y_true, y_pred):.4f}")
print(f"RMSE:      {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
print(f"MAE:       {mean_absolute_error(y_true, y_pred):.4f}")

## Technique 2: RestNet

In [ ]:
train_loader = DataLoader(TensorDataset(X_train_torch, y_train_torch), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_torch, y_test_torch), batch_size=64, shuffle=False)

In [ ]:
class ResBlock1D(nn.Module):
    def __init__(self, channels):
        super(ResBlock1D, self).__init__()
        self.conv = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.bn(self.conv(x)))
        out = self.bn(self.conv(out))
        return self.relu(out + residual)

class ResNet1D(nn.Module):
    def __init__(self, input_dim, seq_len):
        super(ResNet1D, self).__init__()
        self.init_conv = nn.Conv1d(input_dim, 32, kernel_size=3, padding=1)
        self.res_layers = nn.Sequential(ResBlock1D(32), ResBlock1D(32))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * seq_len, 1)

    def forward(self, x):
        # x shape: (Batch, Seq, Features) -> needs (Batch, Features, Seq) for Conv1d
        x = x.transpose(1, 2) 
        x = torch.relu(self.init_conv(x))
        x = self.res_layers(x)
        x = self.flatten(x)
        return self.fc(x)

In [ ]:
resnet_model = ResNet1D(input_dim=9, seq_len=24).to(device)
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("Training 1D-CNN ResNet on Power Dataset...")
for epoch in range(15):
    resnet_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        loss = criterion(resnet_model(batch_X).squeeze(), batch_y)
        loss.backward()
        optimizer.step()

In [ ]:
resnet_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        preds = resnet_model(batch_X)
        y_pred.extend(preds.squeeze().tolist())
        y_true.extend(batch_y.tolist())

y_true, y_pred = np.array(y_true), np.array(y_pred)

print("\n--- 1D-CNN ResNet Metrics (Electric Power) ---")
print(f"R^2 Score: {r2_score(y_true, y_pred):.4f}")
print(f"MSE:       {mean_squared_error(y_true, y_pred):.4f}")
print(f"RMSE:      {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
print(f"MAE:       {mean_absolute_error(y_true, y_pred):.4f}")